## Name : Vedant Shirgaonkar
## D114 | 60009230002

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
df = pd.read_csv('movies_metadata.csv')
df = df.iloc[:10000, :]
df.head()

/var/folders/q6/0pfb75k55856k8_ms3tldx240000gn/T/ipykernel_59331/2485397617.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [29]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [30]:
#Compute the mean vote average from the dataset and store it as C.
C = df['vote_average'].mean()


In [31]:
#Calculate the 90th percentile of vote_count and store it as m.
m = df['vote_count'].quantile(0.90)
m

381.0

In [32]:
#Filter the movies that have a vote count >= m and store them in a new DataFrame q_movies.
q_movies = df.copy().loc[df['vote_count'] >= m]

In [33]:
#Write a function that computes the weighted rating of each movie using IMDB formula.
def weighted_rating(df, m=m, C=C):
    v = df['vote_count']
    R = df['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [34]:
#Add a new column called score to the q_movies DataFrame, where each movie's score is calculated using the weighted_rating() function.
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#Sort movies based on score calculated above, and Print the top 20 movies.
q_movies = q_movies.sort_values('score', ascending=False)
q_movies.head(20)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
314,False,NaN,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,51.645403,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 9...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-09-23,2.834147e+07,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0,8.396125
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",41.109264,/rPdtLWNsZmAtoZl9PK7S2wE3qiS.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1972-03-14,2.450664e+08,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,8.358273
2843,False,NaN,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1999-10-15,1.008538e+08,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678.0,8.217331
292,False,NaN,8000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,680,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",140.950236,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-09-10,2.139288e+08,154.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670.0,8.208125
522,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",http://www.schindlerslist.com/,424,tt0108052,en,Schindler's List,The true story of how businessman Oskar Schind...,41.725123,/yPisjyLweCl1tbgwgtzBCNCBle.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1993-11-29,3.213656e+08,195.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,"Whoever saves one life, saves the world entire.",Schindler's List,False,8.3,4436.0,8.127369
5481,False,NaN,15000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",http://movies.disney.com/spirited-away,129,tt0245429,ja,千と千尋の神隠し,A ten year old girl who wanders away from her ...,41.048867,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,"[{'name': 'Studio Ghibli', 'id': 10342}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2001-07-20,2.749251e+08,125.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,The tunnel led Chihiro to a mysterious town...,Spirited Away,False,8.3,3968.0,8.108792
351,False,NaN,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,13,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-07-06,6.779454e+08,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8.2,8147.0,8.106958
2211,False,NaN,20000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,637,tt0118799,it,La vita è bella,A touching story of an Italian book seller of ...,39.39497,/f7DImXDebOs148U4uPjI61iDvaK.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'

## TF-IDF:

In [35]:
#Print overviews of the first 5 movies.
q_movies['overview'].head()

314     Framed in the 1940s for the double murder of h...
834     Spanning the years 1945 to 1955, a chronicle o...
2843    A ticking-time-bomb insomniac and a slippery s...
292     A burger-loving hit man, his philosophical par...
522     The true story of how businessman Oskar Schind...
Name: overview, dtype: object

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Use TfidfVectorizer to convert the movie descriptions (e.g., from overview column) 
# into TF-IDF vectors. Remove all english stop words such as 'the', 'a'.
tfidf = TfidfVectorizer(stop_words='english')
#Handle NaN values in text data (e.g., replace them with empty strings).
q_movies['overview'] = q_movies['overview'].fillna('')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(q_movies['overview'])


In [40]:
#Output the shape of tfidf_matrix.
tfidf_matrix.shape

(1001, 9013)

In [41]:
#Display an array mapping from feature integer indices to feature name. (Indices from 5000 till 5009).
tfidf.get_feature_names_out()[5000:5010]

array(['marcus', 'mare', 'marge', 'marginalization', 'maria', 'mariachi',
       'marian', 'mariana', 'marie', 'marijuana'], dtype=object)

In [42]:
#Import linear kernel
from sklearn.metrics.pairwise import linear_kernel
#Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.00685295 0.00781084 ... 0.00974468 0.03120342 0.00709296]
 [0.00685295 1.         0.         ... 0.         0.         0.03303067]
 [0.00781084 0.         1.         ... 0.0111409  0.00870147 0.00810924]
 ...
 [0.00974468 0.         0.0111409  ... 1.         0.01085582 0.0380474 ]
 [0.03120342 0.         0.00870147 ... 0.01085582 1.         0.00790174]
 [0.00709296 0.03303067 0.00810924 ... 0.0380474  0.00790174 1.        ]]


In [ ]:
#Create a reverse lookup map from movie titles to their index positions in the metadata DataFrame. Now you can retrieve the index of a movie by its title.
#This index can then be used to fetch similarity scores, metadata, etc.
indices = pd.Series(q_movies.index, index=q_movies['title']).drop_duplicates()
print(indices)


title
The Shawshank Redemption     314
The Godfather                834
Fight Club                  2843
Pulp Fiction                 292
Schindler's List             522
                            ... 
Spy Kids 3-D: Game Over     6417
Daredevil                   6019
Speed 2: Cruise Control     1486
Catwoman                    7939
Batman & Robin              1491
Length: 1001, dtype: int64


In [45]:
q_movies.shape

(1001, 25)

In [47]:
#Build the Recommender Function:
#Create a function that:
# Accepts a movie title.
# Finds its index.
# Fetches similarity scores.  Creates pairs of (index, similarity_score) and converts it into a list of tuples.
# Sorts and returns the top 10 most similar movies.

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return q_movies['title'].iloc[movie_indices]


In [52]:
#Test the recommender function with a movie title.
print(get_recommendations('Pulp Fiction'))

4845                    Vanilla Sky
332     What's Eating Gilbert Grape
1567           The Devil's Advocate
1909                          Bambi
8927        Kiki's Delivery Service
2211              Life Is Beautiful
3631                    The Patriot
1185                    Raging Bull
1311             Jingle All the Way
7938           The Bourne Supremacy
Name: title, dtype: object
